# EfficientNet on CIFAR-10

In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import sys

sys.path.append('../../')
sys.path.append('../../src/')

import src.general as general
import src.interfaces.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune


In [10]:
model = torch.load('/workspace/volume/models/efficientnet_cifar10.pth')

In [12]:
torch.save(model, '/workspace/volume/models/efficientnet_cifar10.pt')

In [13]:
# Get dataset
dataset = data.supported_datasets["CIFAR-10"]

In [4]:
# Load efficient net model from torchvision
model = tv.models.efficientnet_b7(pretrained=True)
model.classifier[1]


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
general.finetune(model, dataset, target=99, max_it=20, save_path='/workspace/volume/models/efficientnet_cifar10.pth')

Train: 100%|██████████| 11250/11250 [18:18<00:00, 10.24it/s]


Train loss: 0.5209
Train score: 83.1511


Validate: 100%|██████████| 79/79 [00:17<00:00,  4.43it/s]


Test loss: 0.1978
Test score: 95.0356


Train: 100%|██████████| 11250/11250 [18:53<00:00,  9.92it/s]


...

Train: 100%|██████████| 11250/11250 [18:57<00:00,  9.89it/s]


Train loss: 0.0183
Train score: 99.4289


Validate: 100%|██████████| 79/79 [00:17<00:00,  4.42it/s]

Test loss: 0.1437
Test score: 96.1432
Finetuning stopped due to early stopping with patience = 3
Finetuning finished after 11 iterations
Best score: 97.0530


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormAct

In [6]:
pruned_model = prune.channel_pruning(model, dataset, 0.5, fineTune=True, iterative_steps=10)

Train: 100%|██████████| 11250/11250 [18:59<00:00,  9.87it/s]


Train loss: 0.4744
Train score: 84.8133


Train: 100%|██████████| 11250/11250 [19:00<00:00,  9.87it/s]


Train loss: 0.2787
Train score: 90.7733


Train: 100%|██████████| 11250/11250 [19:00<00:00,  9.87it/s]


...

Train loss: 0.0691
Train score: 97.7133


Train: 100%|██████████| 11250/11250 [19:00<00:00,  9.86it/s]


Train loss: 0.0630
Train score: 97.9156


Train: 100%|██████████| 11250/11250 [19:00<00:00,  9.86it/s]

Train loss: 0.0433
Train score: 98.5400


In [12]:
pruend_model = torch.load('/workspace/volume/models/efficientnet_cifar10_pruned.pth')

In [13]:
pruned_results = eval.get_results(pruned_model, dataset)
plot.print_results(**pruned_results)

Test: 100%|██████████| 157/157 [00:35<00:00,  4.46it/s]


Test loss: 0.2439
Test score: 93.7301
==================================== RESULTS ===================================
Loss: 0.243884
Score: 93.730096
Time per data point: 14.0044 ms
Model Size: 244.96 MB
Number of parameters: 63812570
Number of MACs: 5264522736


In [9]:
model = torch.load('/workspace/volume/models/efficientnet_cifar10.pth')

In [10]:
before_prune_results = eval.get_results(model, dataset)
plot.print_before_after_results(before_prune_results, pruned_results)

Test: 100%|██████████| 157/157 [00:35<00:00,  4.46it/s]


Test loss: 0.1356
Test score: 96.1286
============================= RESULTS BEFORE & AFTER ===========================
Loss: 0.135635 -> 0.242822 (79.03%)
Score: 96.128583 -> 93.759952 (-2.46%)
Time per data point: 14.0198 ms -> 14.0267 ms (0.05%)
Model Size: 244.96 MB -> 244.96 MB (-0.00%)
Number of parameters: 63812570 -> 63812570 (-0.00%)
Number of MACs: 5264522736 -> 5264522736 (-0.00%)


In [7]:
torch.save(pruned_model, '/workspace/volume/models/efficientnet_cifar10_pruned.pth')